In [ ]:
# HYPER PARAM
PROJECT_NAME = "RobocupTrajectoryPrediction_DataExpanded"
MODEL_NAME = "BiGRU_dataExpanded_5"
GROUP_NAME = "BiGRU"
BATCH_SIZE = 512
EPOCHS = 100
HIDDEN_DIM = 1024
NUM_LAYER = 16
LEARNING_RATE = 0.001

In [ ]:
import joblib
import os

import visualizer

from tqdm.notebook import tqdm
import datasets
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
import torchmetrics
import pytorch_lightning as pl

In [ ]:
torch.set_float32_matmul_precision("high")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
class MinMax:
    def __init__(self, min, max):
        self.min = min
        self.max = max
        if self.min >= self.max:
            raise ValueError("min must be less than max")

    def __call__(self, x):
        return (x - self.min) / (self.max - self.min)

    def inverse(self, x):
        return x * (self.max - self.min) + self.min

    def __repr__(self):
        return f"MinMax({self.min}, {self.max})"

In [ ]:
train = np.load("datas/train.npy")
cols = np.load("datas/cols.npy", allow_pickle=True)
min_max_d = np.load("datas/min_max_d.npy", allow_pickle=True).item()
test = np.load("datas/test.npy")

In [ ]:
def f():
    df = pd.DataFrame(train[0], columns=cols)
    for i in df.columns:
        df[i] = min_max_d[i].inverse(df[i])
    display(df)
    display(visualizer.plot(df))


# f()

In [ ]:
X_train, y_train = train[:, :20, :], train[:, 20:, :]
X_test, y_test = test[:, :20, :], test[:, 20:, :]

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
for index in torch.cat([torch.tensor([0]), torch.arange(5, 201, 9)]):
    print(index, cols[index])

In [ ]:
class EuclideanDistance(torchmetrics.Metric):
    def __init__(self, cols, min_max_d, **kwargs):
        super().__init__(**kwargs)
        self.cols = cols
        self.min_max_d = min_max_d
        self.add_state("sum", default=torch.tensor(0.0), dist_reduce_fx="sum")
        self.add_state("count", default=torch.tensor(0), dist_reduce_fx="sum")
        self._mins = torch.tensor(
            [min_max_d[col].min for col in cols], dtype=torch.float32
        )
        self._maxs = torch.tensor(
            [min_max_d[col].max for col in cols], dtype=torch.float32
        )
        self._scale = self._maxs - self._mins

    def update(self, preds, target):
        indices = torch.cat([torch.tensor([0]), torch.arange(5, 201, 9)])
        final_preds = preds[:, -1, :]
        final_target = target[:, -1, :]

        inversed_preds = torch.zeros_like(final_preds)
        inversed_target = torch.zeros_like(final_target)

        mins = self._mins.to(device=device, dtype=final_preds.dtype)
        scale = self._scale.to(device=device, dtype=final_preds.dtype)

        inversed_preds = final_preds.clone()
        inversed_target = final_target.clone()
        n_cols = len(self.cols)
        inversed_preds[:, :n_cols] = final_preds[:, :n_cols] * scale + mins
        inversed_target[:, :n_cols] = final_target[:, :n_cols] * scale + mins

        errors = torch.sqrt(
            (inversed_preds[:, indices] - inversed_target[:, indices]) ** 2
            + (inversed_preds[:, indices + 1] - inversed_target[:, indices + 1]) ** 2
        )
        self.sum += torch.sum(errors)
        self.count += errors.size(0)

    def compute(self):
        return self.sum / self.count

In [ ]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# TensorDataset と DataLoader の作成
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, num_workers=16
)
val_loader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True, num_workers=16
)

In [1]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger


class LitBiGRU(pl.LightningModule):
    def __init__(
        self, input_dim, hidden_dim, output_dim, num_layers, seq_length, lr=0.001
    ):
        super().__init__()
        self.save_hyperparameters()
        self.gru = nn.GRU(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
        )
        self.fc = nn.Linear(hidden_dim * 2, output_dim * seq_length)
        self.criterion = nn.MSELoss()
        self.euclidean_distance = EuclideanDistance(cols, min_max_d)
        self.train_losses = []
        self.val_losses = []

    def forward(self, x):
        out, _ = self.gru(x)
        last_out = out[:, -1, :]
        output = self.fc(last_out)
        return output.view(-1, 30, self.hparams.output_dim)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("train_loss", loss, on_step=False, on_epoch=True)
        self.log(
            "train_euclidean_distance",
            self.euclidean_distance(y_hat, y),
            on_step=False,
            on_epoch=True,
        )
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("val_loss", loss, on_step=False, on_epoch=True)
        self.log(
            "val_euclidean_distance",
            self.euclidean_distance(y_hat, y),
            on_step=False,
            on_epoch=True,
        )
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.lr)
        return optimizer


# チェックポイントのコールバックを定義（val_euclidean_distanceが改善したときに保存）
checkpoint_callback = ModelCheckpoint(
    monitor="val_euclidean_distance",
    dirpath=f"checkpoints/{MODEL_NAME}",
    filename="bilstm-{epoch:02d}-{val_euclidean_distance:.4f}",
    save_top_k=3,
    mode="min",
)

wandb_logger = WandbLogger(
    project=PROJECT_NAME,
    log_model=True,
    save_code=True,
    save_dir="logs/",
    name=MODEL_NAME,
    group=GROUP_NAME,
)


trainer = pl.Trainer(
    max_epochs=EPOCHS,
    accelerator="gpu",
    devices=1,
    callbacks=[checkpoint_callback],
    logger=wandb_logger,
)


X_train.shape  # b, 20, 49
y_train.shape  # b, 30, 49

input_dim = X_train.shape[2]
hidden_dim = HIDDEN_DIM
output_dim = y_train.shape[2]
seq_length = y_train.shape[1]
num_layers = NUM_LAYER
learning_rate = LEARNING_RATE

NameError: name 'pl' is not defined

In [12]:
model = LitBiGRU(
    input_dim, hidden_dim, output_dim, num_layers, seq_length, lr=learning_rate
)
trainer.fit(model, train_loader, val_loader)

NameError: name 'exit' is not defined

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f3728286990>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f3aaafeeed0, execution_count=12 error_before_exec=None error_in_exec=name 'exit' is not defined info=<ExecutionInfo object at 7f3aaafef8d0, raw_cell="model = LitBiGRU(
    input_dim, hidden_dim, outpu.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://attached-container%2B7b22636f6e7461696e65724e616d65223a222f6f68617368695f73616e64626f785f31227d@ssh-remote%2B7b22686f73744e616d65223a224136303030227d/root/robocup/v0_learn_gru_data_expand.ipynb#X14sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
import time

best_model_path = checkpoint_callback.best_model_path
model = LitBiGRU.load_from_checkpoint(best_model_path)
file_name = f"{MODEL_NAME}_{time.strftime('%Y%m%d%H%M%S')}.pth"

os.makedirs(f"models/{MODEL_NAME}", exist_ok=True)
torch.save(model.state_dict(), f"models/{MODEL_NAME}/{file_name}")

In [ ]:
import visualizer
from importlib import reload

reload(visualizer)
import visualizer


def visualize_test(X, y, index=0):

    X_test, y_test = X[index, :, :], y[index, :, :]
    X_test = np.array([X_test])

    X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

    model.to(device)

    with torch.no_grad():
        pred = model.forward(X_test)

    pred = pred[0]
    X_test = X_test[0]

    X_test = X_test.cpu().numpy()
    pred = pred.cpu().numpy()
    y_test = y_test.cpu().numpy()

    pred = np.concatenate([X_test, pred], axis=0)
    y_test = np.concatenate([X_test, y_test], axis=0)
    for index, col in enumerate(cols):
        pred[:, index] = min_max_d[col].inverse(pred[:, index])
        y_test[:, index] = min_max_d[col].inverse(y_test[:, index])

    return visualizer.plot_np2(y_test, pred, cols)


import pathlib

video_filepaths = []
for i in range(10):
    anim = visualize_test(X_test, y_test, index=i)
    video_filename = f"{MODEL_NAME}_{i}.gif"
    video_dir = pathlib.Path("videos")
    video_dir.mkdir(exist_ok=True)
    anim.save(video_dir / video_filename, writer="pillow", fps=10)
    video_filepaths.append(video_dir / video_filename)

wandb_logger.log_video("example", [str(i) for i in video_filepaths])

In [ ]:
euclidean_distance = EuclideanDistance(cols, min_max_d).to(device)

error = 0
model.eval()
for X, y in tqdm(val_loader):
    X, y = X.to(device), y.to(device)
    with torch.no_grad():
        pred = model.forward(X)
    error += euclidean_distance(pred, y)
print(error / len(val_loader))

In [ ]:
import visualizer
from importlib import reload

reload(visualizer)
import visualizer


def f(X, y, index=0):
    X_test, y_test = X[index, :, :], y[index, :, :]
    X_test = np.array([X_test])

    X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

    model.to(device)

    with torch.no_grad():
        pred = model.forward(X_test)

    pred = pred[0]
    X_test = X_test[0]

    X_test = X_test.cpu().numpy()
    pred = pred.cpu().numpy()
    y_test = y_test.cpu().numpy()

    pred = np.concatenate([X_test, pred], axis=0)
    y_test = np.concatenate([X_test, y_test], axis=0)
    for index, col in enumerate(cols):
        pred[:, index] = min_max_d[col].inverse(pred[:, index])
        y_test[:, index] = min_max_d[col].inverse(y_test[:, index])

    return visualizer.plot_np2(y_test, pred, cols)


display(f(X_test, y_test, index=0))

In [ ]:
def col2index(cols):
    d = {}

    for i, col in enumerate(cols):
        if col.endswith("_x"):
            if col[:-2] in d:
                d[col[:-2]][0] = i
            else:
                d[col[:-2]] = [i, None]
        if col.endswith("_y"):
            if col[:-2] in d:
                d[col[:-2]][1] = i
            else:
                d[col[:-2]] = [None, i]
    colx = [d[i][0] for i in d]
    coly = [d[i][1] for i in d]
    colors = []
    for key, value in d.items():
        if key.startswith("b"):
            colors.append("b")
        elif key.startswith("l"):
            colors.append("r")
        elif key.startswith("r"):
            colors.append("g")
    return colx, coly, colors


def f():
    colx, coly, color = col2index(cols)
    for i, (x, y, c) in enumerate(zip(colx, coly, color)):
        print(f"{i}: {cols[x]} - {cols[y]} - {c}")


f()